In [1]:
!pip install torch torchvision torchaudio
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
!pip install opencv-python-headless

Cloning into 'yolov5'...
remote: Enumerating objects: 16517, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 16517 (delta 45), reused 50 (delta 16), pack-reused 16402
Receiving objects: 100% (16517/16517), 15.17 MiB | 15.63 MiB/s, done.
Resolving deltas: 100% (11302/11302), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.0/721.0 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.6 MB/s eta 0:00:00


In [2]:
import os
import shutil
import cv2
import zipfile
from sklearn.model_selection import train_test_split
from skimage import io
from skimage.color import rgba2rgb, gray2rgb
from skimage import img_as_ubyte
from albumentations import Compose, HueSaturationValue, RandomBrightnessContrast


from google.colab import drive
drive.mount('/content/drive')

# Define your paths
zip_path = '/content/drive/MyDrive/Machine Learning Project/training.zip'
dataset_dir = '/content/Dataset'

# Create destination directory if it does not exist and unzip the file
os.makedirs(dataset_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

# Define dataset directories
images_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')

# Function to rotate the images
def rotate_bound(image, angle):
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = abs(M[0, 0])
    sin = abs(M[0, 1])
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    return cv2.warpAffine(image, M, (nW, nH))

# Function to rotate the labels
def rotate_label(label, img_shape, angle):
    h, w = img_shape
    new_label = []
    for line in label:
        parts = line.split()
        class_id, x_center, y_center, width, height = parts[0], float(parts[1]), float(parts[2]), float(parts[3]), float(parts[4])
        x_center, y_center, width, height = x_center * w, y_center * h, width * w, height * h
        if angle == 90:
            x_center, y_center = y_center, w - x_center
        elif angle == 180:
            x_center, y_center = w - x_center, h - y_center
        elif angle == 270:
            x_center, y_center = h - y_center, x_center
        if angle in [90, 270]:
            width, height = height, width
        new_label.append(f"{class_id} {x_center / w if angle in [180, 0] else x_center / h} {y_center / h if angle in [180, 0] else y_center / w} {width / w if angle in [180, 0] else width / h} {height / h if angle in [180, 0] else height / w}")
    return new_label

# Function to augment and normalize images
def augment_and_normalize_image(image):
    if image.ndim == 2:  # Grayscale
        image = gray2rgb(image)
    elif image.ndim == 3 and image.shape[2] == 4:  # RGBA to RGB
        image = rgba2rgb(image)
    image = img_as_ubyte(image)
    transform = Compose([
        HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
        RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5)
    ])
    augmented = transform(image=image)
    return augmented['image']

# Function to process, rotate and move images and labels
def process_and_rotate_images(files, source_images_dir, source_labels_dir, dest_images_dir, dest_labels_dir, augment=False):
    for img_name in files:
        if img_name.endswith('.png'):
            base_name = img_name[:-4]
            img_path = os.path.join(source_images_dir, img_name)
            label_path = os.path.join(source_labels_dir, base_name + '.txt')
            image = cv2.imread(img_path)
            with open(label_path, 'r') as file:
                original_label = file.readlines()

            # Augment and save the original image and label in destination directory if specified
            if augment:
                image = augment_and_normalize_image(image)
            new_img_path = os.path.join(dest_images_dir, img_name)
            new_label_path = os.path.join(dest_labels_dir, base_name + '.txt')
            cv2.imwrite(new_img_path, image)
            with open(new_label_path, 'w') as file:
                for line in original_label:
                    file.write(f"{line}\n")

            # Process for each angle
            for angle in [90, 180, 270]:
                rotated_img = rotate_bound(image, angle)
                rotated_label = rotate_label(original_label, image.shape[:2], angle)
                new_img_path = os.path.join(dest_images_dir, f"{base_name}_{angle}.png")
                new_label_path = os.path.join(dest_labels_dir, f"{base_name}_{angle}.txt")
                cv2.imwrite(new_img_path, rotated_img)
                with open(new_label_path, 'w') as file:
                    for line in rotated_label:
                        file.write(f"{line}\n")

# Get a list of all original image files
original_images = [img for img in os.listdir(images_dir) if img.endswith('.png')]

# Split the original dataset into training and validation sets
train_images, val_images = train_test_split(original_images, test_size=0.2, random_state=42)

# Define new training and validation directories
train_dir_images = os.path.join(dataset_dir, 'train_images')
train_dir_labels = os.path.join(dataset_dir, 'train_labels')
val_dir_images = os.path.join(dataset_dir, 'val_images')
val_dir_labels = os.path.join(dataset_dir, 'val_labels')

# Create directories if they don't exist
os.makedirs(train_dir_images, exist_ok=True)
os.makedirs(train_dir_labels, exist_ok=True)
os.makedirs(val_dir_images, exist_ok=True)
os.makedirs(val_dir_labels, exist_ok=True)

# Process, rotate, and move the training and validation images and labels
process_and_rotate_images(train_images, images_dir, labels_dir, train_dir_images, train_dir_labels, augment=True)
process_and_rotate_images(val_images, images_dir, labels_dir, val_dir_images, val_dir_labels)


Mounted at /content/drive


In [3]:
import os
import shutil

# Your original dataset directory
dataset_dir = '/content/Dataset'

# New directory structure
new_dirs = {
    'train_images': 'train/images',
    'train_labels': 'train/labels',  # Note: Consider correcting 'lables' to 'labels' if it's a typo in your directory names.
    'val_images': 'val/images',
    'val_labels': 'val/labels'  # Same here for 'lables'.
}

# Create new folder structure and move files
for old_dir, new_dir in new_dirs.items():
    # Create new folders
    new_dir_path = os.path.join(dataset_dir, new_dir)
    if not os.path.exists(new_dir_path):
        os.makedirs(new_dir_path)

    # Move files
    old_dir_path = os.path.join(dataset_dir, old_dir)
    for filename in os.listdir(old_dir_path):
        src_file = os.path.join(old_dir_path, filename)
        shutil.move(src_file, new_dir_path)

# Remove old folders
for old_dir in new_dirs.keys():
    old_dir_path = os.path.join(dataset_dir, old_dir)
    shutil.rmtree(old_dir_path)


In [5]:
!python train.py --img 640 --epochs 30 --data /content/Dataset/dataset.yaml --weights yolov5x.pt


2024-03-11 02:10:04.960923: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 02:10:04.960969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 02:10:04.962553: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5x.pt, cfg=, data=/content/Dataset/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, sin

In [6]:
!pip install torch torchvision torchaudio
%pip install ultralytics
import ultralytics
ultralytics.checks()
!pip install torch torchvision torchaudio
!pip install opencv-python-headless
# YOLO v8 training
!yolo train model=yolov8x.pt data=/content/Dataset/dataset.yaml epochs=30 imgsz=640

Ultralytics YOLOv8.1.26 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 39.0/201.2 GB disk)
100% 131M/131M [00:00<00:00, 355MB/s]
Ultralytics YOLOv8.1.26 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/Dataset/dataset.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visu

# Performance Comparison among YOLOv5, YOLOv5 Small, and YOLOv8

This table summarizes the performance metrics for the largest models of YOLOv5, YOLOv5 Small, and YOLOv8 across different categories:

| Metric               | YOLOv5 Large  | YOLOv5 Small  | YOLOv8        |
| -------------------- | ------------- | ------------- | ------------- |
| Number of Layers     | 322           | 157           | 268           |
| Number of Parameters | 86,186,872    | 7,018,216     | 68,126,457    |
| GFLOPs               | 203.8         | 15.8          | 257.4         |
| Overall Precision (P)| 0.873         | 0.88          | 0.87          |
| Overall Recall (R)   | 0.847         | 0.84          | 0.844         |
| Overall mAP50        | 0.902         | 0.905         | 0.905         |
| Overall mAP50-95     | 0.626         | 0.628         | 0.638         |
| Room Precision (P)   | 0.926         | 0.918         | 0.924         |
| Room Recall (R)      | 0.896         | 0.895         | 0.89          |
| Room mAP50           | 0.947         | 0.946         | 0.946         |
| Room mAP50-95        | 0.663         | 0.661         | 0.666         |
| Window Precision (P) | 0.839         | 0.853         | 0.85          |
| Window Recall (R)    | 0.82          | 0.816         | 0.804         |
| Window mAP50         | 0.871         | 0.879         | 0.876         |
| Window mAP50-95      | 0.498         | 0.501         | 0.51          |
| Door Precision (P)   | 0.854         | 0.868         | 0.835         |
| Door Recall (R)      | 0.825         | 0.81          | 0.837         |
| Door mAP50           | 0.886         | 0.89          | 0.894         |
| Door mAP50-95        | 0.718         | 0.723         | 0.739         |

From the updated comparison table, we can observe the following:

1. **Model Complexity and Computational Cost**: The small version of YOLOv5 offers a significant reduction in both the number of layers and parameters compared to its larger counterpart and YOLOv8. It also requires substantially less computational power (15.8 GFLOPs).

2. **Performance (Precision, Recall, mAP)**:
   - The small version of YOLOv5 shows competitive performance compared to the larger models, particularly in overall precision and mAP50.
   - Despite its smaller size, YOLOv5 Small performs comparably to or slightly better than YOLOv5 Large in several metrics and is competitive with YOLOv8, especially considering its lower computational requirements.

This data suggests that YOLOv5 Small could be a highly efficient model for environments with stringent computational or storage limitations while still maintaining high levels of accuracy.
